<a href="https://colab.research.google.com/github/funkdub/Pytorch/blob/master/Pytorch_MNIST_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [218]:
print('Importing ...')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets


import os
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Importing ...


使用datasets 准备数据集，并且将训练集 验证机 以及 测试集准备好。

In [219]:
# 其实就是数据增强操作,在dataloader中使用transforms
data_transforms = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
]
)

train_data = datasets.MNIST('data',train=True,download=True,transform=data_transforms)
test_data = datasets.MNIST('data',train=False,download=True,transform=data_transforms)

n_train = int(len(train_data)*0.9)
n_valid = len(train_data) - n_train

train_data,valid_data = torch.utils.data.random_split(train_data,[n_train,n_valid])

print('train_data num is',n_train,'|valid_data numm is ',n_valid,'|test_data numm is',len(test_data))

train_data num is 54000 |valid_data numm is  6000 |test_data numm is 10000


参数

In [0]:
BATCH_SIZE = 64

train_data_it = data.DataLoader(train_data, shuffle=True, batch_size = BATCH_SIZE)
valid_data_it = data.DataLoader(valid_data, batch_size = BATCH_SIZE)
test_data_it = data.DataLoader(test_data, batch_size = BATCH_SIZE)


定义Model

In [0]:
class MLP(nn.Module):
  def __init__(self,n_hidden):
    super(MLP,self).__init__()
    
    self.n_hidden = n_hidden
    
    self.in_hidden = nn.Linear(28*28 , n_hidden[0])
    self.hiddens = nn.ModuleList([nn.Linear(n_hidden[i],n_hidden[i+1]) for i in range(len(n_hidden)-1)])
    self.hidden_out = nn.Linear(n_hidden[-1],10)
   
  def forward(self,x):
    x = x.view(x.shape[0],-1)
    x = F.relu(self.in_hidden(x))
    
    for i in range(len(self.n_hidden) - 1):
      x = F.relu(self.hiddens[i](x))
    x = self.hidden_out(x)
    return x


In [222]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = MLP([1000,500,250]).to(device)
optimizer = optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()


cuda


计算准确率

In [0]:
def accuracy_cal(pred, y):
  # tensor.max选取 tensor中的最大值 
  # max(1)将 最大值的 index 作为结果输出
  prediction = pred.max(1)[1]
  correct = prediction.eq(y.view_as(prediction)).sum()
  acc = correct.float() / prediction.shape[0]
  return acc

In [0]:
def train(model, device , data , optimizer , loss_func):
  epoch_loss = 0
  epoch_acc = 0
  
  model.train()
  
  for (x,y) in data:
    
    x = x.to(device)
    y = y.to(device)

    optimizer.zero_grad()
    pred = model(x)
    loss = loss_func(pred, y)
    acc = accuracy_cal(pred,y)
    loss.backward()
    optimizer.step()
    
    # item()将单值tensor转换成标量
    epoch_loss += loss.item()
    epoch_acc += acc.item()
  return epoch_loss / len(data), epoch_acc / len(data)

In [0]:
def test(model,device,data,loss_func):
  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  
  with torch.no_grad():
    for (x,y) in data:
      x = x.to(device)
      y = y.to(device)
      
      pred = model(x)
      loss = loss_func(pred,y)
      acc = accuracy_cal(pred,y)
      
      epoch_loss += loss
      epoch_acc += acc
  return epoch_loss / len(data), epoch_acc / len(data)
      

In [226]:
EPOCH = 10
SAVE_DIR = 'models'
MODEL_SAVE_DIR = os.path.join(SAVE_DIR,'mlp_mnist.pt')

best_valid_loss = float('inf')


if not os.path.isdir(f'{SAVE_DIR}'):
  os.makedirs(f'{SAVE_DIR}')
  
for epoch in range(EPOCH):
  print('Epoch is ',epoch)
  train_loss , train_acc = train(model,device,train_data_it,optimizer,loss_func)
  valid_loss , valid_acc = test(model,device,valid_data_it,loss_func)
  
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(),MODEL_SAVE_DIR)
  print('|EPOCH ',epoch,'|Train Loss is ',train_loss,'|Train Acc is ',train_acc,'|Valid Loss is ',valid_loss,'|Valid acc is ',valid_acc)

Epoch is  0
|EPOCH  0 |Train Loss is  0.22025260271866456 |Train Acc is  0.9316498815165877 |Valid Loss is  tensor(0.1524, device='cuda:0') |Valid acc is  tensor(0.9533, device='cuda:0')
Epoch is  1
|EPOCH  1 |Train Loss is  0.09853652892501857 |Train Acc is  0.970471712085308 |Valid Loss is  tensor(0.0982, device='cuda:0') |Valid acc is  tensor(0.9699, device='cuda:0')
Epoch is  2
|EPOCH  2 |Train Loss is  0.06966805823535716 |Train Acc is  0.9785680786410779 |Valid Loss is  tensor(0.1111, device='cuda:0') |Valid acc is  tensor(0.9674, device='cuda:0')
Epoch is  3
|EPOCH  3 |Train Loss is  0.05355143110528235 |Train Acc is  0.9835480845652486 |Valid Loss is  tensor(0.1065, device='cuda:0') |Valid acc is  tensor(0.9707, device='cuda:0')
Epoch is  4
|EPOCH  4 |Train Loss is  0.04674166278461611 |Train Acc is  0.9858745556872038 |Valid Loss is  tensor(0.0894, device='cuda:0') |Valid acc is  tensor(0.9759, device='cuda:0')
Epoch is  5
|EPOCH  5 |Train Loss is  0.03737597504048056 |Train A

In [231]:
model.load_state_dict(torch.load(MODEL_SAVE_DIR))
test_loss, test_acc = test(model,device,test_data_it,loss_func)
print('Test Loss is ',test_loss.item(),'|Test ACC is ',test_acc.item())

Test Loss is  0.07583176344633102 |Test ACC is  0.9790008068084717
